In [1]:
%matplotlib inline

from gym_yahtzee.envs.yahtzee_env import YahtzeeSingleEnv

import gym

from collections import namedtuple
from itertools import count
import math
import numpy as np
import random
from statistics import mean, median
from typing import Any, List, NamedTuple, Tuple

import matplotlib
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# if gpu is to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
# set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display
    
plt.ion()

In [3]:
"""Calculate mean and median total reward for random actions"""
env: YahtzeeSingleEnv = gym.make('yahtzee-single-v0')
episode_count = 1000
max_round_count = 1000
input_size = 0

rewards: List[int] = []
for episode in range(episode_count):
    env.reset()

    if episode == 0:
        state = env.get_observation_space()
        input_size = len(state)
        print('Example state for one round:')
        print(state)
    episode_reward = 0
    for round in range(max_round_count):
        action = env.sample_action()
        state, reward, done, info = env.step(action)
        episode_reward += reward
        if done:
            rewards.append(episode_reward)
#            print(f'Episode {episode + 1} finished after {round + 1} timesteps, score: {episode_reward}')
            break

print(f'Mean reward after {episode_count} episodes: {mean(rewards)}, ' \
      f'median: {median(rewards)}, min: {min(rewards)}, max: {max(rewards)}')

Example state for one round:
(0, 1, 6, 6, 1, 6, 2, 1, 1, 0, 0, 0, 3, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1)
Mean reward after 1000 episodes: 46.907, median: 43.0, min: 13, max: 136


In [4]:
"""Base classes for Reinforcement Learning model"""
class Transition(NamedTuple):
    state: Tuple[Any, ...]  # TODO: these are not tuples, but Tensors
    action: int
    next_state: Tuple[Any, ...]
    reward: int


class ReplayMemory(object):

    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.position = 0

    def push(self, *args):
        """Saves a transition."""
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        self.memory[self.position] = Transition(*args)
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

class DQN(nn.Module):
    def __init__(self, input_size):
        super(DQN, self).__init__()
        self.l1 = nn.Linear(in_features = input_size, out_features = 128)
        self.l2 = nn.Linear(in_features = 128, out_features = 128)
        self.l3 = nn.Linear(in_features = 128, out_features = 44)

    def forward(self, x):
        x = x.view(-1, 28)
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        return self.l3(x)
#        return self.l3(x.view(x.size(0), -1))
#        return F.softmax(self.l3(x), dim = 0)
#        return self.l3(x.view(x.size(0), -1))


In [5]:
BATCH_SIZE = 128
GAMMA = 0.999
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 200
TARGET_UPDATE = 10

policy_net = DQN(input_size).to(device)
target_net = DQN(input_size).to(device)
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()

optimizer = optim.RMSprop(policy_net.parameters())
memory = ReplayMemory(10000)


steps_done = 0

In [6]:
def select_action(state):
    global steps_done
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * math.exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1
    if sample > eps_threshold:
        with torch.no_grad():
            # t.max(1) will return largest column value of each row.
            # second column on max result is index of where max element was
            # found, so we pick action with the larger expected reward.
            return policy_net(state).max(1)[1].view(1, 1)
    else:
        return torch.tensor([[random.randrange(44)]], device=device, dtype=torch.long)


episode_durations = []


def plot_durations():
    plt.figure(2)
    plt.clf()
    durations_t = torch.tensor(episode_durations, dtype=torch.float)
    plt.title('Training...')
    plt.xlabel('Episode')
    plt.ylabel('Duration')
    plt.plot(durations_t.numpy())
    # Take 100 episode averages and plot them too
    if len(durations_t) >= 100:
        means = durations_t.unfold(0, 100, 1).mean(1).view(-1)
        means = torch.cat((torch.zeros(99), means))
        plt.plot(means.numpy())

    plt.pause(0.001)  # pause a bit so that plots are updated
    if is_ipython:
        display.clear_output(wait=True)
        display.display(plt.gcf())

In [7]:
def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
    # detailed explanation). This converts batch-array of Transitions
    # to Transition of batch-arrays.
    batch = Transition(*zip(*transitions))

    # Compute a mask of non-final states and concatenate the batch elements
    # (a final state would've been the one after which simulation ended)
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), device=device, dtype=torch.uint8)
    non_final_next_states = torch.cat([s for s in batch.next_state
                                                if s is not None])
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken. These are the actions which would've been taken
    # for each batch state according to policy_net
    state_action_values = policy_net(state_batch).gather(1, action_batch)

    # Compute V(s_{t+1}) for all next states.
    # Expected values of actions for non_final_next_states are computed based
    # on the "older" target_net; selecting their best reward with max(1)[0].
    # This is merged based on the mask, such that we'll have either the expected
    # state value or 0 in case the state was final.
    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    next_state_values[non_final_mask] = target_net(non_final_next_states).max(1)[0].detach()
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    # Compute Huber loss
    loss = F.smooth_l1_loss(state_action_values, expected_state_action_values.unsqueeze(1))

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    for param in policy_net.parameters():
        param.grad.data.clamp_(-1, 1)
    optimizer.step()

In [ ]:
num_episodes = 1000
for i_episode in range(num_episodes):
    # Initialize the environment and state
    env.reset()
    state = torch.tensor(env.get_observation_space(), dtype=torch.float)
    cum_reward = 0
    for t in count():
        # Select and perform an action
        action = select_action(state)
        next_state, reward, done, _ = env.step(action.item())
        next_state = torch.tensor(next_state, dtype=torch.float)
        cum_reward += reward
        reward = torch.tensor([reward], dtype=torch.float, device=device)

        # Store the transition in memory
        memory.push(state, action, next_state, reward)

        # Move to the next state
        state = next_state

        # Perform one step of the optimization (on the target network)
        optimize_model()
        if done:
            episode_durations.append(cum_reward)
            plot_durations()
            break
    # Update the target network, copying all weights and biases in DQN
    if i_episode % TARGET_UPDATE == 0:
        target_net.load_state_dict(policy_net.state_dict())

print('Complete')
env.render()
env.close()
plt.ioff()
plt.show();

<Figure size 432x288 with 0 Axes>

In [9]:
plt.savefig('a.png')

<Figure size 432x288 with 0 Axes>

In [10]:
episode_durations

[64,
 28,
 39,
 29,
 21,
 64,
 33,
 37,
 54,
 43,
 59,
 102,
 69,
 49,
 66,
 37,
 66,
 47,
 91,
 64,
 18,
 27,
 31,
 29,
 61,
 102,
 29,
 32,
 49,
 36,
 61,
 68,
 30,
 43,
 52,
 63,
 22,
 61,
 54,
 40,
 37,
 51,
 32,
 30,
 52,
 25,
 60,
 29,
 70,
 33]